# Multilinear Regression study on Discretionary Income

1. Pick variables
2. Analyze the data i.e summary statistics
3. Correlation Matrix
4. Create a training & test set
5. Use multilinear regression
6. Assess accuracy

## Extract

In [73]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [63]:
df = pd.read_csv('GE-DMYR-2015-Final-Rates.csv')
df.head()

,Institution Code (six-digit OPEID),Institution Name,City,State,Zip,Institution Type,CIP Code,CIP Name,Credential Level,Official Program Pass/Zone/Fail,...,Debt-to-Earnings Transitional Rate,Debt-to-Earnings Transitional Rate Numerator,Debt-to-Earnings Transitional Rate Denominator,Debt-to-Earnings Transitional Rate Pass/Fail/Zone,Debt-to-Earnings Transitional Discretionary Income Rate,Debt-to-Earnings Transitional Discretionary Income Rate Numerator,Debt-to-Earnings Transitional Discretionary Income Rate Denominator,Debt-to-Earnings Transitional Discretionary Income Rate Pass/Fail/Zone,Mean Annual Earnings From SSA,Median Annual Earnings from SSA
0,1007,CENTRAL ALABAMA COMMUNITY COLLEGE,ALEXANDER CITY,AL,350100000.0,PUBLIC 2 TO 3 YEARS,513901,LICENSED PRACTICAL/VOCATIONAL NURSE TRAINING.,01 - UNDERGRADUATE CERTIFICATE,PASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32617,35061
1,1013,CALHOUN COMMUNITY COLLEGE,TANNER,AL,356710000.0,PUBLIC 2 TO 3 YEARS,513901,LICENSED PRACTICAL/VOCATIONAL NURSE TRAINING.,01 - UNDERGRADUATE CERTIFICATE,PASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26219,31071
2,1017,GADSDEN STATE COMMUNITY COLLEGE,GADSDEN,AL,359032269.0,PUBLIC 2 TO 3 YEARS,120401,"COSMETOLOGY/COSMETOLOGIST, GENERAL.",01 - UNDERGRADUATE CERTIFICATE,PASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12114,11605
3,1017,GADSDEN STATE COMMUNITY COLLEGE,GADSDEN,AL,359032269.0,PUBLIC 2 TO 3 YEARS,480508,WELDING TECHNOLOGY/WELDER.,01 - UNDERGRADUATE CERTIFICATE,PASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29104,29702
4,1017,GADSDEN STATE COMMUNITY COLLEGE,GADSDEN,AL,359032269.0,PUBLIC 2 TO 3 YEARS,510909,SURGICAL TECHNOLOGY/TECHNOLOGIST.,01 - UNDERGRADUATE CERTIFICATE,PASS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21605,23598


## Cleaning Data

In [64]:
# Dependent Variable: 'Debt-to-Earnings Discretionary Income Rate'
# Independent Variables: 'Institution Name', 'City', 'State', 'Zip',
# 'Institution Type', 'CIP Name', 'Credential Level'

# Transforming DataFrame
df = df[['Credential Level', 'Mean  Annual Earnings From SSA', 'Median Annual Earnings from SSA' ]]

# Formatting column as integers
# df['Debt-to-Earnings Discretionary Income Rate'] = pd.to_numeric(df['Debt-to-Earnings Discretionary Income Rate'], 
#                                                                 errors = 'coerce',
#                                                                 downcast = 'integer')

## Transforming Data

In [65]:
# Selecting categorical features
categories = list(df.select_dtypes(include = 'object'))
categories

['Credential Level']

In [66]:
# Creating numerics from our categorical data
for cat in categories:
    print(df[cat].value_counts())

01 - UNDERGRADUATE CERTIFICATE         6082
02 - ASSOCIATES DEGREE                 1464
03 - BACHELORS DEGREE                   598
05 - MASTERS DEGREE                     267
08 - GRADUATE CERTIFICATE               114
04 - POST BACCALAUREATE CERTIFICATE      48
06 - DOCTORAL DEGREE                     47
07 - FIRST PROFESSIONAL DEGREE           17
Name: Credential Level, dtype: int64


In [67]:
# Creating dummy variables to represent our categorical data
df = df.dropna()
df = pd.get_dummies(df, drop_first = True)
df.head()

,Mean Annual Earnings From SSA,Median Annual Earnings from SSA,Credential Level_02 - ASSOCIATES DEGREE,Credential Level_03 - BACHELORS DEGREE,Credential Level_04 - POST BACCALAUREATE CERTIFICATE,Credential Level_05 - MASTERS DEGREE,Credential Level_06 - DOCTORAL DEGREE,Credential Level_07 - FIRST PROFESSIONAL DEGREE,Credential Level_08 - GRADUATE CERTIFICATE
0,32617,35061,0,0,0,0,0,0,0
1,26219,31071,0,0,0,0,0,0,0
2,12114,11605,0,0,0,0,0,0,0
3,29104,29702,0,0,0,0,0,0,0
4,21605,23598,0,0,0,0,0,0,0


In [68]:
# Isolating X and y & adding constant
y = df['Median Annual Earnings from SSA']
X = df.drop(columns = ['Median Annual Earnings from SSA', 'Mean  Annual Earnings From SSA'])
X = sm.add_constant(X)
print(X)

      const  Credential Level_02 - ASSOCIATES DEGREE  \
0       1.0                                        0   
1       1.0                                        0   
2       1.0                                        0   
3       1.0                                        0   
4       1.0                                        0   
...     ...                                      ...   
8632    1.0                                        0   
8633    1.0                                        0   
8634    1.0                                        0   
8635    1.0                                        0   
8636    1.0                                        0   

      Credential Level_03 - BACHELORS DEGREE  \
0                                          0   
1                                          0   
2                                          0   
3                                          0   
4                                          0   
...                                    

## Test & Training Sets

In [69]:
X_test, X_train, y_test, y_train = train_test_split(X, y,
                                                    train_size = 0.2,
                                                    random_state = 1502)

X_train.head()

,const,Credential Level_02 - ASSOCIATES DEGREE,Credential Level_03 - BACHELORS DEGREE,Credential Level_04 - POST BACCALAUREATE CERTIFICATE,Credential Level_05 - MASTERS DEGREE,Credential Level_06 - DOCTORAL DEGREE,Credential Level_07 - FIRST PROFESSIONAL DEGREE,Credential Level_08 - GRADUATE CERTIFICATE
5297,1.0,0,0,0,0,0,0,0
1869,1.0,0,0,0,0,0,0,0
8349,1.0,0,0,0,0,0,0,0
264,1.0,0,0,0,0,0,0,0
3397,1.0,0,1,0,0,0,0,0


## Multilinear Regression

In [70]:
model = sm.OLS(y_train, X_train).fit()
print(model.summary())

                                   OLS Regression Results                                  
Dep. Variable:     Median Annual Earnings from SSA   R-squared:                       0.306
Model:                                         OLS   Adj. R-squared:                  0.306
Method:                              Least Squares   F-statistic:                     435.4
Date:                             Sun, 30 Jul 2023   Prob (F-statistic):               0.00
Time:                                     20:43:10   Log-Likelihood:                -75058.
No. Observations:                             6910   AIC:                         1.501e+05
Df Residuals:                                 6902   BIC:                         1.502e+05
Df Model:                                        7                                         
Covariance Type:                         nonrobust                                         
                                                           coef    std err      

## Assessing Accuracy

In [71]:
predictions = model.predict(X_test)
predictions.head()

7522    21742.725434
3644    49824.862385
6295    21742.725434
1202    25305.541166
8450    21742.725434
dtype: float64

In [74]:
print('MAPE:', round(mean_absolute_percentage_error(y_test, predictions), 1))
print('MAE:', round(mean_absolute_error(y_test, predictions), 1))
print('RSME:', round(np.sqrt(mean_squared_error(y_test, predictions)), 1))

MAPE: 5.669978585009017e+17
MAE: 9630.6
RSME: 13672.8
